In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary
from sklearn.model_selection import train_test_split

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import collections

In [2]:
# GPU Device
gpu_id = 0
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 0: True


# Arguments

In [3]:
data_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN_256/'

In [4]:
pretrained = ''
resume = './log/style1/128/b1/checkpoint.pth.tar'

In [5]:
# Model
model_name = 'efficientnet-b1' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 300
start_epoch = 0
train_batch = 180
test_batch = 180
lr = 0.1
schedule = [20, 125, 225, 275]
momentum = 0.9
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/style1/128/b1' # dir
if not os.path.isdir(checkpoint):
    os.mkdir(checkpoint)
num_workers = 4

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

best_acc = 0

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

# Dataset

In [7]:
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'validation')    
train_aug = transforms.Compose([
    transforms.RandomAffine(degrees=2, translate=(0.02, 0.02), scale=(0.98, 1.02), shear=2, fillcolor=(124,117,104)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(datasets.ImageFolder(val_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [8]:
model = EfficientNet.from_name(model_name, num_classes=num_classes)

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    model.load_state_dict(torch.load(pretrained)['state_dict'])

In [9]:
model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))

    Total params: 6.52M


In [10]:
# summary(model, input_size=(3,64,64), device='cuda')

# Loss

In [11]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=1e-4, nesterov=True)
# optimizer = optim.Adam(model.parameters(), weight_decay=1e-4)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=8, total_epoch=10, after_scheduler=scheduler_cosine)

In [12]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.'])

==> Resuming from checkpoint..


# Train

In [13]:
def train(train_loader, model, criterion, optimizer, epoch, use_cuda):
    model.train()
    torch.set_grad_enabled(True)
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    end = time.time()
    
    bar = Bar('Processing', max=len(train_loader))
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=batch_idx + 1,
                    size=len(train_loader),
                    data=data_time.val,
                    bt=batch_time.val,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,
                    )
        bar.next()
        if batch_idx % 10 == 0:
            print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [14]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()
    torch.set_grad_enabled(False)

    end = time.time()
    bar = Bar('Processing', max=len(val_loader))
    for batch_idx, (inputs, targets) in enumerate(val_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs, volatile=True), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:} | top1: {top1:}'.format(
                    batch=batch_idx + 1,
                    size=len(val_loader),
                    data=data_time.avg,
                    bt=batch_time.avg,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,)
        bar.next()
    print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
         batch=batch_idx+1, size=len(val_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [15]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [16]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    train_loss, train_acc = train(train_loader, model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_acc = test(val_loader, model, criterion, epoch, use_cuda)
    
    logger.append([state['lr'], train_loss, test_loss, train_acc, test_acc])
    scheduler_warmup.step()

    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : model.state_dict(),
        'acc': test_acc,
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)


Epoch: [179 | 300] LR: 0.008000
1/383 Data:1.954 | Batch:9.293 | Total:0:00:09 | ETA:0:59:11 | Loss:0.03689337521791458 | top1:98.33333587646484
11/383 Data:0.002 | Batch:0.695 | Total:0:00:17 | ETA:0:10:41 | Loss:0.030046518384055657 | top1:98.58586120605469
21/383 Data:0.002 | Batch:0.688 | Total:0:00:24 | ETA:0:04:12 | Loss:0.03383557904245598 | top1:98.59788513183594
31/383 Data:0.002 | Batch:0.689 | Total:0:00:31 | ETA:0:04:02 | Loss:0.03325541877758599 | top1:98.74551391601562
41/383 Data:0.002 | Batch:0.687 | Total:0:00:38 | ETA:0:03:59 | Loss:0.03414642053270122 | top1:98.73983764648438
51/383 Data:0.001 | Batch:0.686 | Total:0:00:45 | ETA:0:03:53 | Loss:0.03406592981670709 | top1:98.70370483398438
61/383 Data:0.001 | Batch:0.690 | Total:0:00:52 | ETA:0:03:41 | Loss:0.03551129537985706 | top1:98.60655975341797
71/383 Data:0.001 | Batch:0.698 | Total:0:00:59 | ETA:0:03:38 | Loss:0.0363900180525658 | top1:98.61502838134766
81/383 Data:0.002 | Batch:0.677 | Total:0:01:06 | ETA:0:

/home/cutz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


44/44 Data:0.002 | Batch:1.277 | Total:0:00:19 | ETA:0:00:00 | Loss:0.016144387744582045 | top1:99.38461303710938

Epoch: [180 | 300] LR: 0.001700
1/383 Data:2.009 | Batch:2.891 | Total:0:00:02 | ETA:0:18:25 | Loss:0.014124613255262375 | top1:100.00001525878906
11/383 Data:0.002 | Batch:0.696 | Total:0:00:09 | ETA:0:05:41 | Loss:0.031746819615364075 | top1:98.78787994384766
21/383 Data:0.012 | Batch:0.688 | Total:0:00:16 | ETA:0:04:15 | Loss:0.03275326138273591 | top1:98.83598327636719
31/383 Data:0.004 | Batch:0.709 | Total:0:00:23 | ETA:0:04:06 | Loss:0.031240845760030132 | top1:98.90680694580078
41/383 Data:0.001 | Batch:0.715 | Total:0:00:30 | ETA:0:03:57 | Loss:0.030062764860326198 | top1:98.95663452148438
51/383 Data:0.001 | Batch:0.708 | Total:0:00:37 | ETA:0:03:56 | Loss:0.030960630862882324 | top1:98.92156982421875
61/383 Data:0.003 | Batch:0.698 | Total:0:00:44 | ETA:0:03:47 | Loss:0.031258044733864364 | top1:98.8797836303711
71/383 Data:0.002 | Batch:0.695 | Total:0:00:51 | 

301/383 Data:0.001 | Batch:0.714 | Total:0:03:32 | ETA:0:00:57 | Loss:0.033150522312941494 | top1:98.80767059326172
311/383 Data:0.002 | Batch:0.706 | Total:0:03:39 | ETA:0:00:51 | Loss:0.033447275976605496 | top1:98.79420471191406
321/383 Data:0.001 | Batch:0.702 | Total:0:03:46 | ETA:0:00:44 | Loss:0.03312294602008229 | top1:98.80409240722656
331/383 Data:0.002 | Batch:0.564 | Total:0:03:54 | ETA:0:00:38 | Loss:0.03293686260587432 | top1:98.806640625
341/383 Data:0.001 | Batch:0.715 | Total:0:04:01 | ETA:0:00:30 | Loss:0.03285282205782088 | top1:98.79927825927734
351/383 Data:0.001 | Batch:0.726 | Total:0:04:08 | ETA:0:00:23 | Loss:0.03289316270428781 | top1:98.80342102050781
361/383 Data:0.001 | Batch:0.690 | Total:0:04:15 | ETA:0:00:16 | Loss:0.03281281127754362 | top1:98.8057861328125
371/383 Data:0.001 | Batch:0.687 | Total:0:04:22 | ETA:0:00:09 | Loss:0.03265811619725369 | top1:98.81401062011719
381/383 Data:0.017 | Batch:0.672 | Total:0:04:29 | ETA:0:00:02 | Loss:0.032587100896

211/383 Data:0.001 | Batch:0.690 | Total:0:02:30 | ETA:0:02:01 | Loss:0.03255612568673788 | top1:98.76251220703125
221/383 Data:0.013 | Batch:0.698 | Total:0:02:37 | ETA:0:01:53 | Loss:0.032423297014947124 | top1:98.77073669433594
231/383 Data:0.002 | Batch:0.698 | Total:0:02:44 | ETA:0:01:51 | Loss:0.0326661590539177 | top1:98.77345275878906
241/383 Data:0.024 | Batch:0.750 | Total:0:02:51 | ETA:0:01:40 | Loss:0.032896108629715266 | top1:98.76441192626953
251/383 Data:0.022 | Batch:0.732 | Total:0:02:58 | ETA:0:01:34 | Loss:0.032613247679360954 | top1:98.76936340332031
261/383 Data:0.001 | Batch:0.631 | Total:0:03:05 | ETA:0:01:26 | Loss:0.0331372498798673 | top1:98.74627685546875
271/383 Data:0.001 | Batch:0.637 | Total:0:03:12 | ETA:0:01:21 | Loss:0.03318790015266225 | top1:98.73513793945312
281/383 Data:0.003 | Batch:0.789 | Total:0:03:19 | ETA:0:01:13 | Loss:0.03323002727006254 | top1:98.72676849365234
291/383 Data:0.017 | Batch:0.721 | Total:0:03:26 | ETA:0:01:05 | Loss:0.0331822

...

331/383 Data:0.005 | Batch:0.723 | Total:0:03:55 | ETA:0:00:36 | Loss:0.025966150212745967 | top1:99.04833984375
341/383 Data:0.003 | Batch:0.678 | Total:0:04:02 | ETA:0:00:31 | Loss:0.025998963394729405 | top1:99.05180358886719
351/383 Data:0.011 | Batch:0.709 | Total:0:04:09 | ETA:0:00:23 | Loss:0.02592563525073103 | top1:99.05191802978516
361/383 Data:0.002 | Batch:0.704 | Total:0:04:16 | ETA:0:00:16 | Loss:0.02579094358549616 | top1:99.05817413330078
371/383 Data:0.011 | Batch:0.716 | Total:0:04:23 | ETA:0:00:09 | Loss:0.0258613432389797 | top1:99.05660247802734
381/383 Data:0.000 | Batch:0.716 | Total:0:04:30 | ETA:0:00:02 | Loss:0.02604110058203946 | top1:99.04928588867188
44/44 Data:0.002 | Batch:0.113 | Total:0:00:15 | ETA:0:00:00 | Loss:0.010789717078352204 | top1:99.5769271850586

Epoch: [294 | 300] LR: 0.000059
1/383 Data:2.394 | Batch:3.081 | Total:0:00:03 | ETA:0:19:37 | Loss:0.047236569225788116 | top1:98.8888931274414
11/383 Data:0.001 | Batch:0.718 | Total:0:00:10 | ETA

241/383 Data:0.012 | Batch:0.712 | Total:0:02:51 | ETA:0:01:42 | Loss:0.025942780379847433 | top1:99.02951049804688
251/383 Data:0.001 | Batch:0.732 | Total:0:02:58 | ETA:0:01:34 | Loss:0.025789091006418564 | top1:99.03939819335938
261/383 Data:0.003 | Batch:0.717 | Total:0:03:05 | ETA:0:01:26 | Loss:0.02582958461163687 | top1:99.03788757324219
271/383 Data:0.001 | Batch:0.764 | Total:0:03:12 | ETA:0:01:19 | Loss:0.025819355469863737 | top1:99.04264068603516
281/383 Data:0.003 | Batch:0.705 | Total:0:03:19 | ETA:0:01:14 | Loss:0.02607687128777367 | top1:99.03321075439453
291/383 Data:0.001 | Batch:0.724 | Total:0:03:26 | ETA:0:01:05 | Loss:0.026185669199342735 | top1:99.02061462402344
301/383 Data:0.013 | Batch:0.734 | Total:0:03:34 | ETA:0:01:00 | Loss:0.02609455091584833 | top1:99.02731323242188
311/383 Data:0.001 | Batch:0.693 | Total:0:03:41 | ETA:0:00:51 | Loss:0.0261148382494843 | top1:99.02465057373047
321/383 Data:0.002 | Batch:0.672 | Total:0:03:48 | ETA:0:00:44 | Loss:0.02631

151/383 Data:0.012 | Batch:0.696 | Total:0:01:49 | ETA:0:02:42 | Loss:0.024919174044140128 | top1:99.09124755859375
161/383 Data:0.002 | Batch:0.699 | Total:0:01:56 | ETA:0:02:36 | Loss:0.024552135288669084 | top1:99.10973358154297
171/383 Data:0.013 | Batch:0.744 | Total:0:02:03 | ETA:0:02:31 | Loss:0.02470890691024605 | top1:99.10655975341797
181/383 Data:0.002 | Batch:0.709 | Total:0:02:10 | ETA:0:02:23 | Loss:0.024783814405682832 | top1:99.09146881103516
191/383 Data:0.002 | Batch:0.690 | Total:0:02:17 | ETA:0:02:15 | Loss:0.02441101556848637 | top1:99.09249877929688
201/383 Data:0.002 | Batch:0.706 | Total:0:02:24 | ETA:0:02:07 | Loss:0.024188208721568856 | top1:99.10724639892578
211/383 Data:0.002 | Batch:0.684 | Total:0:02:31 | ETA:0:02:02 | Loss:0.024735477447324385 | top1:99.09162902832031
221/383 Data:0.001 | Batch:0.708 | Total:0:02:38 | ETA:0:01:54 | Loss:0.024960743822789504 | top1:99.08245849609375
231/383 Data:0.002 | Batch:0.691 | Total:0:02:45 | ETA:0:01:47 | Loss:0.02

61/383 Data:0.002 | Batch:0.712 | Total:0:00:45 | ETA:0:03:48 | Loss:0.030672968311815478 | top1:98.79781341552734
71/383 Data:0.002 | Batch:0.714 | Total:0:00:52 | ETA:0:03:46 | Loss:0.02966654103275546 | top1:98.85758972167969
81/383 Data:0.001 | Batch:0.705 | Total:0:00:59 | ETA:0:03:35 | Loss:0.029164257427723505 | top1:98.88888549804688
91/383 Data:0.001 | Batch:0.690 | Total:0:01:06 | ETA:0:03:25 | Loss:0.028217412492997206 | top1:98.92552185058594
101/383 Data:0.001 | Batch:0.688 | Total:0:01:13 | ETA:0:03:18 | Loss:0.028468232078588392 | top1:98.89988708496094
111/383 Data:0.001 | Batch:0.665 | Total:0:01:20 | ETA:0:03:10 | Loss:0.02825402996402133 | top1:98.90390014648438
121/383 Data:0.013 | Batch:0.745 | Total:0:01:27 | ETA:0:03:04 | Loss:0.028501969891069227 | top1:98.90266418457031
131/383 Data:0.001 | Batch:0.704 | Total:0:01:34 | ETA:0:02:59 | Loss:0.02824777613557477 | top1:98.93553924560547
141/383 Data:0.001 | Batch:0.730 | Total:0:01:41 | ETA:0:02:54 | Loss:0.0279821

371/383 Data:0.006 | Batch:0.682 | Total:0:04:21 | ETA:0:00:09 | Loss:0.025722455585405552 | top1:99.05210876464844
381/383 Data:0.002 | Batch:0.705 | Total:0:04:28 | ETA:0:00:02 | Loss:0.025701720599363757 | top1:99.05074310302734
44/44 Data:0.002 | Batch:0.083 | Total:0:00:17 | ETA:0:00:00 | Loss:0.010260273503641096 | top1:99.5641098022461
